# Kriging

In this preamble, we load the **gstlearn** library.

In [ ]:
import gstlearn as gl
import gstlearn.plot as gp
import matplotlib.pyplot as plt
import numpy as np
import os

Then we download the data base **dat**.

In [ ]:
fileNF = os.path.join(os.getenv('GSTLEARN_DATA'), "Scotland", "Scotland_Temperatures.NF")
dat = gl.Db.createFromNF(fileNF)

Calculate the experimental variogram **vario2dir** (in 2 directions)

In [ ]:
varioParamMulti = gl.VarioParam.createMultiple(2, 2, 15, 15.)
vario2dir = gl.Vario(varioParamMulti, dat)
err = vario2dir.compute()

Calculate the fitted model **fitmodOK** (add the Universality Condition)

In [ ]:
fitmodOK = gl.Model()
err = fitmodOK.fit(vario2dir,types=[gl.ECov.NUGGET, gl.ECov.EXPONENTIAL, gl.ECov.GAUSSIAN])
err = fitmodOK.addDrift(gl.Drift1())
ax = gp.varmod(vario2dir, fitmodOK)

Define the Unique Neighborhood **unique.neigh**.

In [ ]:
uniqueNeigh = gl.NeighUnique.create(ndim = 2)

Get the extension of the Data:

In [ ]:
dat.getExtremas()

Create the Target file **grid**.

In [ ]:
grid = gl.DbGrid.create(x0=[65,535],dx=[4.94, 4.96],nx=[81,137])
dbfmt = gl.DbStringFormat.createFromFlags(flag_resume=False, flag_vars=True,
                                          flag_extend=True, flag_stats=False,
                                          flag_array=False)
grid.display(dbfmt)

Perform Ordinary Kriging

In [ ]:
err = gl.kriging(dat,grid,fitmodOK,uniqueNeigh,namconv=gl.NamingConvention("OK"))

Plottong the results

In [ ]:
ax = gp.grid(grid, title="Ordinary Kriging over whole Grid", figsize=[8,8])

Reading the Elevation Grid

In [ ]:
fileNF = os.path.join(os.getenv('GSTLEARN_DATA'), "Scotland", "Scotland_Elevations.NF")
grid = gl.DbGrid.createFromNF(fileNF)
grid.display(dbfmt)

The output grid now contains the selection **inshore**. 
Estimation is restricted to the active cells only.

In [ ]:
err = gl.kriging(dat,grid,fitmodOK,uniqueNeigh,namconv=gl.NamingConvention("OK"))

Plotting the Estimation

In [ ]:
ax = gp.grid(grid,name="OK*estim",title="Estimation by Ordinary Kriging", figsize=[8,8])
ax = gp.point(dat,size_name="January_temp",ax=ax)

Plotting the Standard deviation of Estimation Error

In [ ]:
ax = gp.grid(grid,name="OK*stdev",title="St. dev. by Ordinary Kriging", figsize=[8,8])
ax = gp.point(dat,size_name="January_temp",ax=ax)

The Model **fitmodOK** is first duplicated into **fitmodSK**. Then the Universality Condition is deleted.

In [ ]:
fitmodSK = fitmodOK.clone()
err = fitmodSK.delDrift(0)
err = fitmodSK.setMean(0, 20.)

Simple Kriging is performed

In [ ]:
err = gl.kriging(dat,grid,fitmodSK,uniqueNeigh, namconv=gl.NamingConvention("SK"))

Plotting the Estimation

In [ ]:
ax = gp.grid(grid,name="SK*estim",title="Estimation by Simple Kriging", figsize=[8,8])
ax = gp.point(dat,size_name="January_temp",ax=ax)

Plotting the Standard deviation of the Estimation error

In [ ]:
ax = gp.grid(grid,name="SK*stdev",title="St. dev. by Simple Kriging", figsize=[8,8])
ax = gp.point(dat,size_name="January_temp",ax=ax)

Comparing Ordinary and Simple Kriging - Estimations

In [ ]:
ax = gp.correlation(grid,namex="OK*estim",namey="SK*estim", bissLine=True,
                 title="Estimation Simple vs. Ordinary", 
                 xlab="Ordinary Kriging", ylab="Simple Kriging")

Comparing Ordinary and Simple Kriging - St. dev.

In [ ]:
ax = gp.correlation(grid,namex = "OK*stdev",namey="SK*stdev", bissLine=True, 
                 title="St. dev. Simple vs. Ordinary", 
                 xlab="Ordinary Kriging", ylab="Simple Kriging")

Cross-Validation

In [ ]:
err = gl.xvalid(dat,fitmodOK,uniqueNeigh, 
             namconv=gl.NamingConvention("Xvalid", True, True, gl.ELoc.UNKNOWN))

Cross-validation - Histogram of Errors

In [ ]:
ax = gp.hist(dat,name="*esterr*",nbins=30,fill="blue",xlab="Estimation Errors",
          title="Cross-Validation")

Cross-validation - Histogram of Standardized Errors

In [ ]:
ax = gp.hist(dat,name="*stderr*",nbins=30,fill="blue",xlab="Standardized Errors",
          title="Cross-Validation")

Cross-validation Scores

In [ ]:
print(np.nanmean(dat.getColumn("*esterr*")))
print(np.nanmean(np.square(dat.getColumn("*esterr*"))))
print(np.nanmean(np.square(dat.getColumn("*stderr*"))))

Display Errors of the Cross Validation

In [ ]:
ax = gp.grid(grid,name="inshore",flagColorBar=False, figsize=[8,8])
ax = gp.point(dat,size_name="*esterr",ax=ax,sizmax=300, title="Cross-Validation scores")

Display Errors of the Cross Validation (in absolute value)

In [ ]:
ax = gp.grid(grid,name="inshore",flagColorBar=False, figsize=[8,8])
ax = gp.point(dat,size_name="*esterr",ax=ax,sizmax=300,
           flagAbsSize=True,title="Cross-Validation scores (abs. value)")

We design a small Moving Neighborhood **small.neigh** with only 1 sample per neighborhood.

In [ ]:
smallNeigh = gl.NeighMoving.create(ndim=2, flag_xvalid=False, nmini=1, nmaxi=1, radius=1000000)

We perform Ordinary Kriging

In [ ]:
err = gl.kriging(dat,grid,fitmodOK,smallNeigh,namconv=gl.NamingConvention("Small"))

Graphic representation

In [ ]:
ax = gp.grid(grid,name="Small*estim",
               title="Estimation by Ordinary Kriging (Small Moving Neigh.)", figsize=[8,8])
ax = gp.point(dat,size_name="January_temp",ax=ax)

Building a reasonable Moving Neighborhood, although with a limited extension (*radius*)

In [ ]:
movingNeigh = gl.NeighMoving.create(ndim=2, nmini=1, nmaxi=10, radius=20)

Running the Ordinary Kriging

In [ ]:
err = gl.kriging(dat,grid,fitmodOK,movingNeigh, namconv=gl.NamingConvention("Reduced"))

Plotting the results

In [ ]:
ax = gp.grid(grid,name="Reduced*estim",
               title="Estimation by Ordinary Kriging (Reduced Moving Neigh.)", figsize=[8,8])
ax = gp.point(dat,size_name="January_temp",ax=ax)

Lots of target sites are not estimated as no sample is found within the neighborhood.

Building a reasonable Moving Neighborhood correctly tuned: 10 samples (maximum) selected in a radius of 150 around the target site.

In [ ]:
movingNeigh = gl.NeighMoving.create(ndim=2, nmini=1, nmaxi=10, radius=150)

Running the Ordinary Kriging

In [ ]:
err = gl.kriging(dat,grid,fitmodOK,movingNeigh, namconv=gl.NamingConvention("Moving"))

Graphic representation

In [ ]:
ax = gp.grid(grid,name="Moving*estim",
               title="Estimation by Ordinary Kriging (Moving Neigh.)", figsize=[8,8])
ax = gp.point(dat,size_name="January_temp",ax=ax)

For the standard deviation of Estimation error

In [ ]:
ax = gp.grid(grid,name="Moving*stdev",
               title="St. dev. by Ordinary Kriging (Moving Neigh.)", figsize=[8,8])
ax = gp.point(dat,size_name="January_temp",ax=ax)

Comparing Unique and Moving Neighborhoods: Estimations

In [ ]:
ax = gp.correlation(grid,namex = "OK*estim",namey="Moving*estim", bissLine=True, 
                 title="Unique vs. Moving Neighborhood", 
                 xlab="Unique", ylab="Moving")

Comparing Unique and Moving Neighborhoods: Standard deviations

In [ ]:
ax = gp.correlation(grid,namex = "OK*stdev",namey="Moving*stdev", bissLine=True, 
                 title="Unique vs. Moving Neighborhood", 
                 xlab="Unique", ylab="Moving")